## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


In [ ]:

!pip install "SQLAlchemy>=2,<3" --quiet

In [ ]:
from sqlalchemy import create_engine, select, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# se o .db está na mesma pasta do notebook, esse caminho funciona:
engine = create_engine("sqlite:///chinook.db", echo=False)

Base = automap_base()
Base.prepare(autoload_with=engine)  # reflete as tabelas existentes

print("Tabelas mapeadas:", list(Base.classes.keys()))

#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

In [ ]:
def get_model(Base, *candidates):
    for name in candidates:
        try:
            return getattr(Base.classes, name)
        except AttributeError:
            pass
    raise AttributeError(f"Não encontrei nenhuma dessas classes: {candidates}")

def col(model, *candidates):
    cols = model.__table__.c
    for name in candidates:
        if name in cols:
            return cols[name]
    raise KeyError(f"Coluna não encontrada em {model.__name__}: {candidates}")

# tabelas (aceita plural/singular dependendo da versão do Chinook)
Tracks       = get_model(Base, "tracks", "Track")
Albums       = get_model(Base, "albums", "Album")
Artists      = get_model(Base, "artists", "Artist")
InvoiceItems = get_model(Base, "invoice_items", "InvoiceLine", "InvoiceItems", "invoiceline")

# colunas (variam entre camel case e snake case)
t_id       = col(Tracks, "track_id", "TrackId")
t_name     = col(Tracks, "name", "Name")
t_album_id = col(Tracks, "album_id", "AlbumId")

al_id      = col(Albums, "album_id", "AlbumId")
al_title   = col(Albums, "title", "Title")
al_art_id  = col(Albums, "artist_id", "ArtistId")

ar_id      = col(Artists, "artist_id", "ArtistId")
ar_name    = col(Artists, "name", "Name")

ii_track   = col(InvoiceItems, "track_id", "TrackId")
ii_qty     = col(InvoiceItems, "quantity", "Quantity")
ii_price   = col(InvoiceItems, "unit_price", "UnitPrice")
ii_id      = col(InvoiceItems, "invoice_item_id", "InvoiceLineId", "InvoiceItemId")

print("OK! Classes e colunas detectadas.")

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
session = Session(engine)
session

#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [ ]:
### Imprima os três primeiros registros da tabela tracks

stmt = select(Tracks).limit(3)
for row in session.execute(stmt).scalars():
    print(f"TrackId={getattr(row, t_id.key)}, Name={getattr(row, t_name.key)}")

In [ ]:

### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.

# Célula 6 — Nome da faixa + título do álbum (20 primeiras)
stmt = (
    select(t_name, al_title)
    .select_from(Tracks)
    .join(Albums, t_album_id == al_id)
    .limit(20)
)
for name, album_title in session.execute(stmt):
    print(f"{name}  —  {album_title}")

In [ ]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
# Célula 7 — 10 primeiras linhas de venda
stmt = select(InvoiceItems).limit(10)
for ii in session.execute(stmt).scalars():
    print(
        f"ItemId={getattr(ii, ii_id.key)} | TrackId={getattr(ii, ii_track.key)} "
        f"| Qty={getattr(ii, ii_qty.key)} | UnitPrice={getattr(ii, ii_price.key)}"
    )


In [ ]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
# Célula 8 — Nome da faixa e quantidade (10 primeiras vendas)
stmt = (
    select(t_name, ii_qty)
    .select_from(InvoiceItems)
    .join(Tracks, ii_track == t_id)
    .limit(10)
)
for name, qty in session.execute(stmt):
    print(f"{name} — Qty {qty}")

In [ ]:

# Célula 9 — Top 10 faixas por quantidade vendida
stmt = (
    select(t_name.label("faixa"), func.sum(ii_qty).label("qtd"))
    .select_from(InvoiceItems)
    .join(Tracks, ii_track == t_id)
    .group_by(t_id, t_name)
    .order_by(func.sum(ii_qty).desc())
    .limit(10)
)
for faixa, qtd in session.execute(stmt):
    print(f"{faixa} — {qtd} vendas")

In [ ]:

### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
# Célula 10 — Top 10 artistas por quantidade vendida
stmt = (
    select(ar_name.label("artista"), func.sum(ii_qty).label("qtd"))
    .select_from(InvoiceItems)
    .join(Tracks, ii_track == t_id)
    .join(Albums, t_album_id == al_id)
    .join(Artists, al_art_id == ar_id)
    .group_by(ar_id, ar_name)
    .order_by(func.sum(ii_qty).desc())
    .limit(10)
)
for artista, qtd in session.execute(stmt):
    print(f"{artista} — {qtd} vendas")